# Youtube Food Recipes to Text

In [1]:
# !pip install pytubefix
# !pip install pydub

In [2]:
import torch

from huggingface_hub import login
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor,pipeline
from diffusers import DiffusionPipeline
from datasets import load_dataset

from openai import OpenAI
from IPython.display import Markdown, display, update_display, Audio
from dotenv import load_dotenv

import nest_asyncio
import soundfile as sf
import os
from pytubefix import YouTube
from pydub import AudioSegment

nest_asyncio.apply()   
load_dotenv(override=True)

login(os.environ['HF_TOKEN'],
      add_to_git_credential=True)
home_dir = os.path.expanduser("~")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
url_list = ["https://www.youtube.com/watch?v=hYFsHfPquPg",
            'https://www.youtube.com/watch?v=TphXloX3PEI',
            'https://www.youtube.com/watch?v=2KHAuPqDYyg'
           ]

use_svr = 'LOCAL'
data_path = f'{home_dir}/Google Drive/My Drive/Projects/Data/Knowledge-base/Unsorted'
data_path_failed_val =  f'{home_dir}/Google Drive/My Drive/Projects/Data/Failed_val'

In [4]:
LOCAL_SVR_MODEL = 'qwen3-14b' #"qwen3-14b-mlx"
LOCAL_SVR_BASE_URL = "http://10.10.10.2:1234/v1"
LOCAL_SVR_API_KEY= 'None'

LOCAL_MODEL = 'qwen3-14b-mlx' #"qwen3-14b-mlx" phi-4-reasoning-plus-mlx
LOCAL_BASE_URL = "http://127.0.0.1:1234/v1"
LOCAL_API_KEY= 'None'

#Config for google LLM
google_api_key = os.getenv('GOOGLE_API_KEY')
google_model = 'gemini-2.0-flash' #Light weight model to gather data
google_base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"

AUDIO_MODEL = "openai/whisper-medium"

VAL_MODEL = 'gemini-2.0-flash' #"qwen3-14b-mlx"
VAL_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
VAL_API_KEY= os.getenv('GOOGLE_API_KEY')

match use_svr:
    case 'GOOGLE':
        MODEL = google_model
        api_key = google_api_key
        base_url = google_base_url
    case 'LOCAL_SVR':
        MODEL = LOCAL_SVR_MODEL
        api_key = LOCAL_SVR_API_KEY
        base_url = LOCAL_SVR_BASE_URL
    case _:
        MODEL = LOCAL_MODEL
        api_key = LOCAL_API_KEY
        base_url = LOCAL_BASE_URL

openai = OpenAI(base_url=base_url,
                api_key=api_key)

In [5]:
system_message = """
                    You are an assistant that produces
                    Detail recipes from transcripts.
                    
                    Recipe title
                    if it available, add in the chef's name, if not remove it. 
                    Include URL link to the video.
                    Executive summary.
                    if it availabel, Total Cooking timing and preparation timing, if not remove it.
                    if it available, Servings PAX, if not remove it, if not remove it. 
                    Detailed Ingredient List.
                    Method of the recipes.
                    Tips and tricks.
                    Conclusion.

                    It also very important to use both metric 
                    and US measurement system. 
                    If there is no measurement in the transcript 
                    say that it is unspecified,
                    don't try to make up any details.
                    
                    It is very important cross check only with the transcript 
                    and if you don't know,
                    don't try to make up any details.
                    
                """

user_prompt = f"""Below is an extract transcript of youtube video. 
                Please write a detail report only based on transcript in markdown.
                
                It is very important be very detailed, 
                cross check with transcript 
                to ensure that report details are correct.
                
                Extracted transcript of youtube video:""" 
                

system_message_val = """
                    You are an assistant that produces
                    that cross check the report with transcript

                    cross check only report's Method 
                    with the transcript
                    it is important to only use transcripts for cross checks. 

                    It is very important to only return a Yes or No answer.
                    Don't include any other verbose explanatiouns, 
                    do not include website in the instruction.
                    and don't include the markdown syntax anywhere. 

                    If the answer is no.
                    It is very important to provide a list of detail of 
                    only what wrong in the report.
                """

user_prompt_val = f""" 
                        cross check only report's Method with the transcript
                        ensure that the report details are correct.

                    """
                


In [6]:
def get_mp3_file(url):
    yt = YouTube(url)
    video = yt.streams.filter(only_audio=True).first()
    downloaded_file = video.download()
    base, ext = os.path.splitext(downloaded_file)
    new_file = 'temp_audio' + '.mp3'
    os.rename(downloaded_file, new_file)
    yt_title = yt.streams[0].title
    
    return new_file, yt_title

def get_transcription(AUDIO_MODEL, audio_file):
    
    
    speech_model = AutoModelForSpeechSeq2Seq\
                        .from_pretrained(AUDIO_MODEL,
                        torch_dtype=torch.float16,
                                         low_cpu_mem_usage=True,
                                         use_safetensors=True,)
    speech_model.to('mps')
    processor = AutoProcessor.from_pretrained(AUDIO_MODEL)
    
    pipe = pipeline(
        "automatic-speech-recognition",
        model=speech_model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        torch_dtype=torch.float16,
        device='mps',
        return_timestamps=True
    )
    
    result = pipe(audio_file)
    return result["text"]

def get_recipes(transcription, user_prompt, system_message, model, url, yt_title):
    yt_title_t = yt_title.replace('/','').replace('|','')\
                        .replace('*','').replace('  ',' ')\
                        .replace(':', '')
    f = open(f'Data/transcription/{yt_title_t}.txt', 'w')
    f.write(transcription)
    f.close()
    
    messages = [
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_prompt \
                                             + f"Video Link: {url}\n"\
                                             + f"Title: {yt_title}\n"\
                                            + transcription }
              ]
    
    stream = openai.chat.completions.create(model=model,
                                                messages=messages,
                                                stream=True,
                                            temperature=0.0
                                            )

    response = ""
    display_handle = display(Markdown(""), display_id=True)

    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

    return response.split('/think>')[-1]

def save_response_to_txt(response, url, folder=data_path):
    counter = 0
    file_name = ''

    response = response.split('</think>')[-1]
    response += '\n\n'
    
    while (len(file_name) > 200) | (file_name == ''):
        file_name = response.replace('```markdown', '')\
          .replace('```', '').splitlines()[counter]\
        .replace('#', '')\
        .replace(':', '')\
        .replace('*', '').replace("''", '')
        
        counter+= 1
    
    while file_name[0]==" ":
        file_name = file_name[1:]
        
    file_name = file_name.replace('_', '')\
                            .replace('#', '')\
                            .replace(':', '')\
                            .replace('*', '')\
                            .replace('/','')\
                            .replace('|','')
                        
        
    f = open(f'{folder}/{file_name}_{MODEL}.txt', 'w')
    f.write(response.replace('```markdown', '').replace('```', ''))
    f.close()
    
    print(f'Saved: {folder}_{file_name}_{MODEL}.txt')

def summery_validation(response, transcription, 
                       system_message_val,user_prompt_val,
                       model=VAL_MODEL, 
                       base_url = VAL_BASE_URL,
                      api_key = VAL_API_KEY):
    print('VAL')
    
    user_prompt_val += f"""Transcription:
                            {transcription}

                            Report:
                            {response}
                            """
    
    val_openai = OpenAI(base_url=base_url,
                api_key=api_key)

    messages = [
                {"role": "system", "content": system_message_val},
                {"role": "user", "content": user_prompt_val \
                                             + f"Video Link: {url}"\
                                            + transcription }
              ] 
    
    val_stream = val_openai.chat.completions.create(model=model,
                                                    messages=messages,
                                                    stream=True,
                                                    temperature=0.0)

    val_response = ""
    display_handle = display(Markdown(""), display_id=True)

    for chunk in val_stream:
        val_response += chunk.choices[0].delta.content or ''
        val_response = val_response.replace("```","").replace("markdown", "")
        update_display(Markdown(val_response), display_id=display_handle.display_id)

    return val_response.split('/think>')[-1]
    

def create_recipes_summery(url, AUDIO_MODEL, 
                           folder=data_path, see_transcription = True):
    
    new_file, yt_title = get_mp3_file(url)
    transcription = get_transcription(AUDIO_MODEL, new_file)
    
    if see_transcription == True:
        print(transcription)
        
    response = get_recipes(transcription, 
                          user_prompt, 
                          system_message, 
                          MODEL, url, yt_title)
    
    val_response = summery_validation(response, transcription, 
                       system_message_val, user_prompt_val,
                       model=VAL_MODEL, 
                       base_url = VAL_BASE_URL,
                      api_key = VAL_API_KEY)
    print(val_response)
    val_response_if = "".join(val_response.splitlines()).replace(' ','')
    
    if val_response_if == 'Yes':
        save_response_to_txt(response, url, folder)
    
    else:
        val_response = val_response.replace("No", "")
        
        response += f'\n\n Report failed validation due to: \n {val_response}'
        save_response_to_txt(response, url, folder = data_path_failed_val)  



In [7]:
for url in url_list:
    print(url)
    create_recipes_summery(url, 
                           AUDIO_MODEL, 
                           see_transcription = False)
    
    print('\n\n')
    

https://www.youtube.com/watch?v=hYFsHfPquPg


Device set to use mps
/opt/anaconda3/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


space

Okay, let's tackle this query. The user wants a detailed recipe report based on the provided YouTube transcript. First, I need to parse through the transcript carefully.

The video title is "Spicy, Crispy Pork Noodles (脆哨面)" from Guizhou province. The chef isn't mentioned, so I'll omit that part. The URL is given as https://www.youtube.com/watch?v=hYFsHfPquPg.

Next, the executive summary. The main points are that it's a Guizhou dish similar to Sichuan dandan noodles but with crispy pork and lard. The process involves making the pork, using rendered lard for flavor, and a spicy chili powder. It's simplified for home cooking.

Cooking time: The transcript mentions about 20 minutes to render the pork and then around 15 seconds for the honey-water step. Preparation time isn't specified, so I'll note that as unspecified.

Servings: The transcript says 120g pork belly per serving, and for two people they used 240g. So servings are 2.

Ingredients: The pork belly is 120g per serving, which translates to metric and US measurements. They mention salt, soy sauce, oil, honey, water, chili powder (Yunnan or Korean gochugaru), MSG, dark vinegar (Chinese 香醋 or balsamic as substitute), scallions, noodles (80g dried per serving), and rendered lard.

Method: The steps are rendering the pork belly, making the sauce with salt, MSG, soy sauce, vinegar, water, scallions. Boiling noodles, assembling the dish with chili powder and crispy pork, then heating the lard to sizzle over everything.

Tips: Use a wok for rendering, adjust chili spice level, use Chinese vinegar or balsamic if unavailable. Also mention the noodle type as thin alkaline egg noodles, similar to Cantonese wonton.

Conclusion: The dish is flavorful and a good home version of the street food, emphasizing the use of rendered lard for authenticity.

I need to ensure all measurements are converted properly. For example, 120g pork belly per serving is metric; US would be around 4.2 oz. The honey-water step is quick, so timing is important.

Check if any details are missing or assumed. The transcript doesn't mention cooking time for the noodles, just boiling until done (~2 minutes). Also, no specific brand names for ingredients, so keep it general.

Make sure to use both metric and US units where possible. For example, 80g noodles per serving is about 2.8 oz.

Avoid adding any information not in the transcript, like specific brands or cooking times that aren't mentioned. The user emphasized cross-checking with the transcript, so I need to be precise.
</think>

# Spicy Crispy Pork Noodles (脆哨面)  
**Video Link:** [https://www.youtube.com/watch?v=hYFsHfPquPg](https://www.youtube.com/watch?v=hYFsHfPquPg)  

---

## Executive Summary  
This recipe for **Spicy Crispy Pork Noodles (脆哨面)** originates from Guizhou province, China. It features crispy pork belly as the star ingredient, cooked in its own rendered lard and served with a spicy chili powder. The dish is simplified for home cooking, using leftover rendered lard to enhance flavor and authenticity. It combines crispy pork, spicy chili powder, and a savory noodle sauce to create a flavorful street food-inspired meal.  

---

## Cooking Time  
- **Total Cooking Time:** ~25 minutes (20 minutes for rendering pork + 15 seconds for honey-water step + ~2 minutes boiling noodles)  
- **Preparation Time:** Unspecified  

---

## Servings (PAX)  
**2 servings**  

---

## Detailed Ingredient List  

### Crispy Pork (Cui Xiao)  
- **Pork belly:** 120 grams per serving (4.2 oz) / Total: **240 grams** (8.5 oz) for 2 servings  
- **Salt:** ¼ teaspoon per serving / Total: **½ teaspoon** for 2 servings  
- **Soy sauce:** 1.5 tablespoons per serving / Total: **3 tablespoons** for 2 servings  
- **Oil:** 2 tablespoons per serving / Total: **4 tablespoons** for 2 servings  

### Noodle Seasoning Sauce  
- **Salt:** 8 teaspoons per serving / Total: **16 teaspoons** for 2 servings  
- **MSG (Monosodium Glutamate):** ¼ teaspoon per serving / Total: **½ teaspoon** for 2 servings  
- **Soy sauce:** 1 tablespoon per serving / Total: **2 tablespoons** for 2 servings  
- **Dark vinegar (Chinese 香醋 or balsamic):** ¾ teaspoon per serving / Total: **1.5 teaspoons** for 2 servings  
- **Water:** 1 tablespoon per serving / Total: **2 tablespoons** for 2 servings  
- **Chopped scallions (green onions):** To taste  

### Additional Ingredients  
- **Crispy pork (Cui Xiao):** As prepared above  
- **Chili powder:** Vibrant red chili powder (e.g., Yunnan Chube or Korean Gochugaru)  
- **Honey:** 1 teaspoon per serving / Total: **2 teaspoons** for 2 servings  
- **Water:** 1 tablespoon per serving / Total: **2 tablespoons** for 2 servings  
- **Noodles:** Thin alkaline egg noodles (e.g., Cantonese wonton noodles) – 80 grams per serving (2.8 oz) / Total: **160 grams** (5.6 oz) for 2 servings  
- **Rendered lard:** From the crispy pork (reserved after straining)  

---

## Method of Preparation  

### Step 1: Prepare Crispy Pork (Cui Xiao)  
1. **Cut pork belly** into ½ cm strips (~0.2 inches).  
2. **Season** with salt and soy sauce, mixing well.  
3. **Coat** the pork strips in oil (2 tablespoons per serving).  
4. **Fry** in a cool wok or pot over medium flame for ~20 minutes, stirring and flipping periodically until golden brown and crispy.  
5. **Strain** the pork and reserve the rendered lard in a separate container.  

### Step 2: Finish Crispy Pork  
1. **Return** the crispy pork to the wok.  
2. **Mix honey** with water (1 teaspoon + 1 tablespoon per serving).  
3. **Sauté over high flame** for ~15 seconds until the water evaporates and the pork is evenly coated.  
4. **Drain** on paper towels to remove excess oil.  

### Step 3: Prepare Noodle Seasoning Sauce  
1. **Combine** all seasoning ingredients (salt, MSG, soy sauce, vinegar, water) in a bowl.  
2. **Chop scallions** and set aside for garnish.  

### Step 4: Cook Noodles  
1. **Boil noodles** (80 grams per serving) in water for ~2 minutes until al dente.  
2. **Drain** and place in serving bowls.  

### Step 5: Assemble the Dish  
1. **Add** noodle seasoning sauce to each bowl of noodles.  
2. **Sprinkle** chili powder, chopped scallions, and crispy pork over the noodles.  
3. **Heat reserved lard** in a small pan until smoking.  
4. **Sizzle the hot lard** over the noodles and toppings to infuse flavor (this step is critical for authenticity).  
5. **Mix thoroughly** until the noodles are slightly sticky and well-coated with lard and seasonings.  

---

## Tips and Tricks  
- **Use a wok** for rendering pork to ensure even cooking and oil separation.  
- **Adjust chili spice level:** Use Korean Gochugaru (1 tablespoon) or a mix of red chili powder and cayenne pepper for heat.  
- **Substitute vinegar:** If Chinese 香醋 is unavailable, use balsamic vinegar for a similar depth of flavor.  
- **Noodle choice:** Use thin alkaline egg noodles (e.g., Cantonese wonton noodles) for authenticity, as they are widely available globally.  
- **Timing is key:** Render pork slowly to avoid burning; monitor closely during the 20-minute process.  

---

## Conclusion  
This **Spicy Crispy Pork Noodles (脆哨面)** recipe captures the essence of Guizhou street food with its crispy pork, spicy chili powder, and aromatic lard. By simplifying the traditional Cui Xiao preparation, it becomes an accessible weeknight dish that balances flavor and texture. The use of rendered lard adds a unique depth, making it a must-try for fans of Sichuan or Guizhou cuisine.

VAL


Yes


...

Device set to use mps
/opt/anaconda3/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


space

Okay, let's start by breaking down the transcript to extract all necessary details for the recipe report. The user wants a detailed  report based solely on the transcript provided.

First, I need to identify the recipe title. The video is titled "Lao Gan Ma Fried Rice (老干妈炒饭)" and mentions a specific shop in Xingyi, Guizhou. The chef isn't explicitly named, so I'll omit that.

Next, the video link is given as https://www.youtube.com/watch?v=2KHAuPqDYyg. I'll include that.

For the executive summary, I should summarize the key points: using Lao Gan Ma chili products, smoked bacon (substituted with American streaky bacon), steamed rice method instead of day-old rice, and the inclusion of cabbage, garlic, ginger, scallions, soy sauce, and canned beans.

Cooking time isn't specified in the transcript, so I'll note that it's unspecified. Servings aren't mentioned either, so I'll leave that out.

Ingredients need to be listed with both metric and US measurements where possible. The transcript mentions 225 grams of jasmine rice, which is about 1 cup (uncooked). The bacon is three pieces cut into strips. Napa cabbage is 40 grams, which might be around a cup chopped. The other ingredients like garlic, ginger, scallions are in quantities that can be converted or left as is if measurements aren't given.

Method steps should follow the transcript closely, detailing each step from steaming rice to frying and adding ingredients. Tips include using day-old rice if available, the bacon rendering technique, and substituting ingredients.

Conclusion should highlight the unique aspects of this recipe compared to others, like using steamed rice and specific Lao Gan Ma products.

I need to ensure that all information is strictly from the transcript, avoiding any assumptions or guesses. If measurements aren't provided, I'll state that they're unspecified.
</think>

# Lao Gan Ma Fried Rice (老干妈炒饭)  
**Video Link:** [https://www.youtube.com/watch?v=2KHAuPqDYyg](https://www.youtube.com/watch?v=2KHAuPqDYyg)  

---

## Executive Summary  
This recipe recreates a variation of **Lao Gan Ma Fried Rice** (老干妈炒饭) inspired by a small shop in Xingyi, Guizhou Province. The dish uses **Lao Gan Ma** chili products (specifically the "big three": chili oil with black bean, fried chili in oil, and spicy chili crisp), **smoked Chinese bacon** (substituted with American-style streaky bacon for international replication), and **steamed rice** instead of day-old rice. The recipe includes Napa cabbage, garlic, ginger, scallions, soy sauce, and canned beans for a savory-tangy flavor profile.  

---

## Cooking Time  
- **Preparation Timing:** Unspecified  
- **Total Cooking Timing:** Unspecified  

---

## Servings PAX  
**Unspecified** (No explicit mention in the transcript)  

---

## Detailed Ingredient List  

### **Base Ingredients**  
- **Jasmine Rice:** 225 grams (about 1.75 cups uncooked)  
- **American Streaky Bacon:** 3 pieces, cut into 1 cm strips (substitute for smoked Chinese bacon)  
- **Napa Cabbage:** 40 grams (about 1 cup, chopped)  

### **Aromatics & Seasonings**  
- **Garlic:** 1 clove, minced  
- **Ginger:** Half a centimeter (about 2 grams), minced  
- **Scallions:** 1 whole, white part minced (for cooking), green part reserved for garnish  
- **Shaoxing Wine:** ½ tablespoon (for aromatics)  
- **Liao Zhuo (Xiaoxing Wine):** ½ tablespoon (for aromatics)  

### **Lao Gan Ma Products**  
- **Red Fragrant Chili Powder:** 2 teaspoons (for heat and flavor)  
- **Lao Gan Ma (Chili Crisp or Oil):** 2 tablespoons (choose based on preference)  

### **Additional Ingredients**  
- **Soy Sauce:** 1 tablespoon (for seasoning)  
- **Canned Beans (any type):** 100 grams, drained and rinsed  

---

## Method of the Recipes  

### **Step 1: Prepare Steamed Rice**  
1. Rinse **225 grams of jasmine rice** 2–3 times under running water.  
2. Cook the rice in a pot of boiling water, uncovered, for **3.5 minutes** or until it loses opacity.  
3. Drain the rice into a strainer, making a small hole in the center for even steaming.  
4. Place the strainer over a pot of boiling water, cover with a lid (sealed with damp rags or aluminum foil), and steam for **15 minutes**.  
5. Remove, toss onto a plate, and set aside.  

---

### **Step 2: Sauté Aromatics**  
1. Heat a wok over high heat, add **1 tablespoon of oil**, swirl to coat the surface.  
2. Add **40 grams of Napa cabbage** strips and stir-fry for **30 seconds** until slightly charred.  
3. Drizzle in **½ tablespoon of Liao Zhuo (Xiaoxing wine)**, mix well, then remove the cabbage.  

---

### **Step 3: Render Bacon**  
1. In the same wok, add **3 pieces of streaky bacon** (cut into strips) and **3 tablespoons of water**.  
2. Cook over high heat, allowing the water to reduce (about **2–3 minutes**).  
3. Reduce flame to medium, add **½ tablespoon of oil**, and fry until bacon is crisp (about **5–6 minutes**).  
4. Scoop the bacon to one side of the wok, drain excess oil if needed.  

---

### **Step 4: Build Flavor Base**  
1. Over medium-low heat, add **minced garlic**, **minced ginger**, and **minced white scallion**.  
2. Sauté for **30 seconds** until fragrant.  
3. Drizzle in **½ tablespoon of Shaoxing wine**, mix well, and let it reduce slightly.  
4. Add **2 teaspoons of red fragrant chili powder**, stir-fry for **30 seconds** until aromatic.  
5. Add **2 tablespoons of Lao Gan Ma** (chili crisp or oil), mix thoroughly.  

---

### **Step 5: Fry the Rice**  
1. Increase heat to high, add the **steamed rice** into the wok.  
2. Toss and fry for **1 minute**, ensuring even coating with the sauce.  
3. Drizzle in **1 tablespoon of soy sauce**, mix well.  

---

### **Step 6: Add Canned Beans & Final Toss**  
1. Add **100 grams of drained/canned beans**, stir-fry for a brief moment to season.  
2. Return the **Napa cabbage** and **crispy bacon** to the wok, toss together.  
3. Turn off heat, add **green scallion tops**, and mix once more before serving.  

---

## Tips and Tricks  
1. **Rice Preparation:** While the video advocates for steamed rice (as used in Guizhou), day-old rice is also acceptable if available.  
2. **Bacon Rendering:** The method of boiling water with bacon and reducing it is a personal preference; traditional frying is also valid.  
3. **Lao Gan Ma Selection:** Use the "big three" products (chili oil with black bean, fried chili in oil, spicy chili crisp) based on desired flavor profile.  
4. **Cabbage Substitution:** If Napa cabbage is unavailable, regular cabbage can be used as a substitute.  
5. **Serving Suggestion:** Pair with a soothing soup (e.g., milky century egg soup) for balance, as mentioned in the transcript.  

---

## Conclusion  
This **Lao Gan Ma Fried Rice** recipe captures the essence of Guizhou’s savory-tangy street food, emphasizing **steamed rice**, **Lao Gan Ma chili products**, and a medley of aromatics like bacon, cabbage, and scallions. The method is adaptable for international kitchens, with substitutions like American bacon and steamed rice ensuring accessibility while staying true to the regional flavor profile. The dish is a flavorful, spicy, and aromatic representation of Guizhou’s culinary traditions.

VAL


Yes


Yes

Saved: /Users/daveng/Google Drive/My Drive/Projects/Data/Knowledge-base/Unsorted_Lao Gan Ma Fried Rice (老干妈炒饭)  _qwen3-14b-mlx.txt



